In [2]:
import glob
source_dir = "/workspace/datasets/product_data/products"
files = glob.glob(source_dir + "/*.xml")
len(files)

256

In [3]:
mappings_old =  [
            "productId/text()", "productId",
            "sku/text()", "sku",
            "name/text()", "name",
            "type/text()", "type",
            "startDate/text()", "startDate",
            "active/text()", "active",
            "regularPrice/text()", "regularPrice",
            "salePrice/text()", "salePrice",
            "artistName/text()", "artistName",
            "onSale/text()", "onSale",
            "digital/text()", "digital",
            "frequentlyPurchasedWith/*/text()", "frequentlyPurchasedWith",# Note the match all here to get the subfields
            "accessories/*/text()", "accessories",# Note the match all here to get the subfields
            "relatedProducts/*/text()", "relatedProducts",# Note the match all here to get the subfields
            "crossSell/text()", "crossSell",
            "salesRankShortTerm/text()", "salesRankShortTerm",
            "salesRankMediumTerm/text()", "salesRankMediumTerm",
            "salesRankLongTerm/text()", "salesRankLongTerm",
            "bestSellingRank/text()", "bestSellingRank",
            "url/text()", "url",
            "categoryPath/*/name/text()", "categoryPath", # Note the match all here to get the subfields
            "categoryPath/*/id/text()", "categoryPathIds", # Note the match all here to get the subfields
            "categoryPath/category[last()]/id/text()", "categoryLeaf",
            "count(categoryPath/*/name)", "categoryPathCount",
            "customerReviewCount/text()", "customerReviewCount",
            "customerReviewAverage/text()", "customerReviewAverage",
            "inStoreAvailability/text()", "inStoreAvailability",
            "onlineAvailability/text()", "onlineAvailability",
            "releaseDate/text()", "releaseDate",
            "shippingCost/text()", "shippingCost",
            "shortDescription/text()", "shortDescription",
            "shortDescriptionHtml/text()", "shortDescriptionHtml",
            "class/text()", "class",
            "classId/text()", "classId",
            "subclass/text()", "subclass",
            "subclassId/text()", "subclassId",
            "department/text()", "department",
            "departmentId/text()", "departmentId",
            "bestBuyItemId/text()", "bestBuyItemId",
            "description/text()", "description",
            "manufacturer/text()", "manufacturer",
            "modelNumber/text()", "modelNumber",
            "image/text()", "image",
            "condition/text()", "condition",
            "inStorePickup/text()", "inStorePickup",
            "homeDelivery/text()", "homeDelivery",
            "quantityLimit/text()", "quantityLimit",
            "color/text()", "color",
            "depth/text()", "depth",
            "height/text()", "height",
            "weight/text()", "weight",
            "shippingWeight/text()", "shippingWeight",
            "width/text()", "width",
            "longDescription/text()", "longDescription",
            "longDescriptionHtml/text()", "longDescriptionHtml",
            "features/*/text()", "features" # Note the match all here to get the subfields

]

In [4]:
mappings = list(zip(mappings_old[1::2], mappings_old[::2]))
mappings[:5]

[('productId', 'productId/text()'),
 ('sku', 'sku/text()'),
 ('name', 'name/text()'),
 ('type', 'type/text()'),
 ('startDate', 'startDate/text()')]

In [7]:
def get_child_mapping(child, mapping):
    return {mapping[0] : child.xpath(mapping[1])}

get_child_mapping(children[0], mappings[0])

{'productId': ['1051806942127']}

In [ ]:
for child in children:
    for mapping in mappings:
        

In [40]:
def make_document(child, mappings):
    return {mapping[0]: child.xpath(mapping[1]) for mapping in mappings}

In [45]:
from lxml import etree
import pandas as pd
tree = etree.parse(files[0])
root = tree.getroot()
children = root.findall("./product")
docs = (make_document(child, mappings) for child in children)

In [43]:
import itertools
list(itertools.islice(docs, 2))

[{'productId': ['1051806941102'],
  'sku': ['12854'],
  'name': ['Manufacturer - Test sku - Color'],
  'type': ['HardGood'],
  'startDate': ['2000-03-01'],
  'active': ['false'],
  'regularPrice': ['3.99'],
  'salePrice': ['3.99'],
  'artistName': [],
  'onSale': ['false'],
  'digital': ['false'],
  'frequentlyPurchasedWith': [],
  'accessories': [],
  'relatedProducts': [],
  'crossSell': [],
  'salesRankShortTerm': [],
  'salesRankMediumTerm': [],
  'salesRankLongTerm': [],
  'bestSellingRank': [],
  'url': [],
  'categoryPath': [],
  'categoryPathIds': [],
  'categoryLeaf': [],
  'categoryPathCount': 0.0,
  'customerReviewCount': [],
  'customerReviewAverage': [],
  'inStoreAvailability': ['false'],
  'onlineAvailability': ['false'],
  'releaseDate': [],
  'shippingCost': [],
  'shortDescription': ['Short Description'],
  'shortDescriptionHtml': ['Short Description'],
  'class': ['FILM'],
  'classId': ['51'],
  'subclass': ['35 MM'],
  'subclassId': ['73'],
  'department': ['PHOTO/C

In [28]:
docs

<generator object <genexpr> at 0x7f9f914deeb0>

In [35]:
child.xpath(mapping[1])

['1051806942127']

In [14]:
len(children)

5000